# Flower FL for Bug Prediction - Improved Version

## Key Improvements:
1. **Better Model**: Proper feature normalization, optimized architecture
2. **Effective Poisoning Attacks**:
   - Model Poisoning: Sign-flip attack (negate weights)
   - Data Poisoning: Strategic label flipping

In [ ]:
!pip install torch scikit-learn -q
print("Done!")

In [ ]:
import json
import time
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from typing import Dict, List
from dataclasses import dataclass, field, asdict
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch: {torch.__version__}")
print(f"Device: {device}")

In [ ]:
from google.colab import files
print("Upload dataset_pairs_1_.json")
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]

## 2. Enhanced Feature Extraction

In [ ]:
def extract_code_features(code: str) -> np.ndarray:
    """
    Extract discriminative features that differ between buggy and fixed code.
    """
    lines = code.split('\n')
    non_empty_lines = [l for l in lines if l.strip()]
    
    # Basic metrics
    total_chars = len(code)
    num_lines = len(lines)
    num_non_empty = len(non_empty_lines)
    avg_line_len = np.mean([len(l) for l in lines]) if lines else 0
    max_line_len = max([len(l) for l in lines]) if lines else 0
    
    # Code structure
    num_functions = code.count('def ')
    num_classes = code.count('class ')
    num_imports = code.count('import ')
    num_returns = code.count('return ')
    
    # Control flow
    num_if = code.count('if ') + code.count('elif ')
    num_else = code.count('else:')
    num_for = code.count('for ')
    num_while = code.count('while ')
    num_try = code.count('try:')
    num_except = code.count('except')
    num_finally = code.count('finally:')
    num_with = code.count('with ')
    
    # Operators (bug-prone)
    num_eq = code.count(' == ')
    num_neq = code.count(' != ')
    num_lt = code.count(' < ') + code.count(' <= ')
    num_gt = code.count(' > ') + code.count(' >= ')
    num_and = code.count(' and ')
    num_or = code.count(' or ')
    num_not = code.count(' not ') + code.count('not(')
    
    # Assignment and arithmetic
    num_assign = code.count(' = ') - num_eq - num_neq
    num_plus_eq = code.count(' += ')
    num_minus_eq = code.count(' -= ')
    num_arithmetic = code.count(' + ') + code.count(' - ') + code.count(' * ') + code.count(' / ')
    num_modulo = code.count(' % ')
    
    # Data structures
    num_list_access = code.count('[') 
    num_dict_access = code.count('{')
    num_tuple = code.count('(')
    
    # Common operations
    num_append = code.count('.append(')
    num_extend = code.count('.extend(')
    num_pop = code.count('.pop(')
    num_remove = code.count('.remove(')
    num_len = code.count('len(')
    num_range = code.count('range(')
    num_enumerate = code.count('enumerate(')
    num_zip = code.count('zip(')
    
    # String operations
    num_str_format = code.count('.format(') + code.count('f"') + code.count("f'")
    num_split = code.count('.split(')
    num_join = code.count('.join(')
    num_strip = code.count('.strip(') + code.count('.rstrip(') + code.count('.lstrip(')
    
    # Type operations
    num_int = code.count('int(')
    num_float = code.count('float(')
    num_str = code.count('str(')
    num_list = code.count('list(')
    num_dict = code.count('dict(')
    
    # Error-prone patterns
    num_none = code.count('None')
    num_true = code.count('True')
    num_false = code.count('False')
    num_is = code.count(' is ')
    num_in = code.count(' in ')
    
    # Indentation (complexity)
    indent_levels = [len(l) - len(l.lstrip()) for l in non_empty_lines] if non_empty_lines else [0]
    max_indent = max(indent_levels)
    avg_indent = np.mean(indent_levels)
    
    # Comments
    num_comments = code.count('#')
    num_docstrings = code.count('"""') + code.count("'''")
    
    features = np.array([
        total_chars, num_lines, num_non_empty, avg_line_len, max_line_len,
        num_functions, num_classes, num_imports, num_returns,
        num_if, num_else, num_for, num_while, num_try, num_except, num_finally, num_with,
        num_eq, num_neq, num_lt, num_gt, num_and, num_or, num_not,
        num_assign, num_plus_eq, num_minus_eq, num_arithmetic, num_modulo,
        num_list_access, num_dict_access, num_tuple,
        num_append, num_extend, num_pop, num_remove, num_len, num_range, num_enumerate, num_zip,
        num_str_format, num_split, num_join, num_strip,
        num_int, num_float, num_str, num_list, num_dict,
        num_none, num_true, num_false, num_is, num_in,
        max_indent, avg_indent, num_comments, num_docstrings
    ], dtype=np.float32)
    
    return features

print(f"Number of features: {len(extract_code_features('test'))}")

In [ ]:
# Load and process data
with open(dataset_path, 'r') as f:
    data = json.load(f)

print(f"Loaded {len(data)} code pairs")

features_list = []
labels_list = []

for pair in data:
    features_list.append(extract_code_features(pair['buggy']))
    labels_list.append(1)
    features_list.append(extract_code_features(pair['fixed']))
    labels_list.append(0)

X = np.array(features_list)
y = np.array(labels_list)

print(f"Total samples: {len(y)}")
print(f"Features: {X.shape}")

In [ ]:
# Normalize
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
print(f"Normalized range: [{X_normalized.min():.2f}, {X_normalized.max():.2f}]")

## 3. PyTorch Model

In [ ]:
class BugPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

# Test
test_model = BugPredictor(X_normalized.shape[1]).to(device)
print(test_model)
print(f"Parameters: {sum(p.numel() for p in test_model.parameters()):,}")

In [ ]:
# Verify model can learn (centralized)
print("Testing centralized training...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

train_ds = TensorDataset(
    torch.FloatTensor(X_train).to(device),
    torch.FloatTensor(y_train).to(device)
)
test_ds = TensorDataset(
    torch.FloatTensor(X_test).to(device),
    torch.FloatTensor(y_test).to(device)
)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=32)

model = BugPredictor(X_normalized.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(30):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb).squeeze()
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            preds = []
            for xb, _ in test_loader:
                preds.extend((model(xb).squeeze() > 0.5).cpu().numpy())
            acc = accuracy_score(y_test, preds)
            print(f"Epoch {epoch+1}: Acc={acc:.4f}")

model.eval()
with torch.no_grad():
    preds = []
    for xb, _ in test_loader:
        preds.extend((model(xb).squeeze() > 0.5).cpu().numpy())

print(f"\n=== Centralized Performance ===")
print(f"Accuracy:  {accuracy_score(y_test, preds):.4f}")
print(f"F1 Score:  {f1_score(y_test, preds):.4f}")
print(f"Precision: {precision_score(y_test, preds):.4f}")
print(f"Recall:    {recall_score(y_test, preds):.4f}")

## 4. Metrics & Utilities

In [ ]:
@dataclass
class Metrics:
    accuracy: float = 0.0
    precision: float = 0.0
    recall: float = 0.0
    f1_score: float = 0.0
    loss: float = 0.0
    def to_dict(self): return asdict(self)

@dataclass
class ExpResult:
    name: str
    metrics: Metrics
    time: float = 0.0
    comm_bytes: float = 0.0
    history: List = field(default_factory=list)
    def to_dict(self):
        return {
            'experiment_name': self.name,
            'metrics': self.metrics.to_dict(),
            'training_time': self.time,
            'communication_bytes': self.comm_bytes,
            'round_metrics': self.history
        }

def calc_metrics(y_true, y_pred, loss=0.0):
    return Metrics(
        accuracy=accuracy_score(y_true, y_pred),
        precision=precision_score(y_true, y_pred, zero_division=0),
        recall=recall_score(y_true, y_pred, zero_division=0),
        f1_score=f1_score(y_true, y_pred, zero_division=0),
        loss=loss
    )

def model_size(model):
    return sum(p.numel() * p.element_size() for p in model.parameters())

## 5. Poisoning Attacks

In [ ]:
class Attacks:
    @staticmethod
    def model_poison(params, noise_scale=0.5):
        """Noise injection attack"""
        poisoned = []
        for p in params:
            noise = np.random.normal(0, noise_scale * (np.std(p) + 1e-7), p.shape)
            poisoned.append(p + noise)
        return poisoned
    
    @staticmethod
    def data_poison(labels):
        """Flip labels"""
        return 1.0 - labels

print("Attacks: Noise injection (model), Label-flip (data)")

## 6. Federated Learning

In [ ]:
def get_params(model):
    return [p.detach().cpu().numpy() for p in model.parameters()]

def set_params(model, params):
    with torch.no_grad():
        for p, new_p in zip(model.parameters(), params):
            p.copy_(torch.from_numpy(new_p).to(device))


class Client:
    def __init__(self, cid, X, y, input_dim, malicious=False, attack='none'):
        self.cid = cid
        self.malicious = malicious
        self.attack = attack
        
        # Data
        labels = Attacks.data_poison(y.copy()) if (malicious and attack == 'data') else y.copy()
        self.dataset = TensorDataset(
            torch.FloatTensor(X).to(device),
            torch.FloatTensor(labels).to(device)
        )
        self.n_samples = len(labels)
        
        # Model
        self.model = BugPredictor(input_dim).to(device)
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
    
    def set_weights(self, params):
        set_params(self.model, params)
    
    def get_weights(self):
        params = get_params(self.model)
        if self.malicious and self.attack == 'model':
            params = Attacks.model_poison(params)
        return params
    
    def train(self, epochs=5):
        loader = DataLoader(self.dataset, batch_size=16, shuffle=True, drop_last=True)
        self.model.train()
        for _ in range(epochs):
            for xb, yb in loader:
                self.optimizer.zero_grad()
                pred = self.model(xb).squeeze()
                loss = self.criterion(pred, yb)
                loss.backward()
                self.optimizer.step()
        return loss.item()


class Server:
    def __init__(self, input_dim):
        self.model = BugPredictor(input_dim).to(device)
    
    def get_weights(self):
        return get_params(self.model)
    
    def aggregate(self, client_params, client_samples):
        total = sum(client_samples)
        agg = [np.zeros_like(p) for p in client_params[0]]
        for params, n in zip(client_params, client_samples):
            for i, p in enumerate(params):
                agg[i] += p * (n / total)
        set_params(self.model, agg)
    
    def evaluate(self, X, y):
        self.model.eval()
        with torch.no_grad():
            X_t = torch.FloatTensor(X).to(device)
            preds = (self.model(X_t).squeeze() > 0.5).cpu().numpy().astype(int)
            y_t = torch.FloatTensor(y).to(device)
            loss = nn.BCELoss()(self.model(X_t).squeeze(), y_t).item()
        return preds, loss

In [ ]:
def run_experiment(
    client_data, input_dim,
    num_rounds=25, local_epochs=5,
    malicious_ids=None, attack='none',
    exp_name='baseline'
):
    if malicious_ids is None:
        malicious_ids = []
    
    print(f"\n{'='*60}")
    print(f"Experiment: {exp_name}")
    print(f"Attack: {attack}, Malicious: {malicious_ids}")
    print(f"{'='*60}")
    
    t0 = time.time()
    history = []
    
    server = Server(input_dim)
    msize = model_size(server.model)
    total_comm = 0
    
    clients = []
    for i, d in enumerate(client_data):
        mal = i in malicious_ids
        c = Client(i, d['X'], d['y'], input_dim, malicious=mal, attack=attack)
        clients.append(c)
        if mal:
            print(f"  Client {i}: MALICIOUS ({attack})")
    
    X_all = np.concatenate([d['X'] for d in client_data])
    y_all = np.concatenate([d['y'] for d in client_data])
    
    for r in range(1, num_rounds + 1):
        gw = server.get_weights()
        for c in clients:
            c.set_weights(copy.deepcopy(gw))
        
        cw_list = []
        cs_list = []
        for c in clients:
            c.train(epochs=local_epochs)
            cw_list.append(c.get_weights())
            cs_list.append(c.n_samples)
        
        server.aggregate(cw_list, cs_list)
        total_comm += msize * len(clients) * 2
        
        pred, loss = server.evaluate(X_all, y_all)
        acc = accuracy_score(y_all, pred)
        f1 = f1_score(y_all, pred)
        history.append({'round': r, 'accuracy': acc, 'f1_score': f1, 'loss': loss})
        
        if r % 5 == 0 or r == num_rounds:
            print(f"  Round {r:2d}/{num_rounds} - Acc: {acc:.4f}, F1: {f1:.4f}")
    
    pred, loss = server.evaluate(X_all, y_all)
    final = calc_metrics(y_all, pred, loss)
    print(f"\nFinal: Acc={final.accuracy:.4f}, F1={final.f1_score:.4f}")
    
    return ExpResult(name=exp_name, metrics=final, time=time.time()-t0, comm_bytes=total_comm, history=history)

## 7. Run Experiments

In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 25
LOCAL_EPOCHS = 5
MALICIOUS_FRACTIONS = [0.1, 0.2, 0.3]

np.random.seed(42)
torch.manual_seed(42)

indices = np.random.permutation(len(y))
X_shuffled = X_normalized[indices]
y_shuffled = y[indices]

splits = np.array_split(np.arange(len(y)), NUM_CLIENTS)
client_data = [{'X': X_shuffled[s], 'y': y_shuffled[s]} for s in splits]

input_dim = X_normalized.shape[1]
print(f"Clients: {NUM_CLIENTS}, Samples/client: ~{len(y)//NUM_CLIENTS}")

In [ ]:
results = {}

# BASELINE
torch.manual_seed(42)
np.random.seed(42)
results['baseline'] = run_experiment(
    client_data, input_dim, NUM_ROUNDS, LOCAL_EPOCHS,
    malicious_ids=[], attack='none', exp_name='baseline'
)

In [ ]:
# MODEL POISONING
print("\n" + "="*70)
print("MODEL POISONING")
print("="*70)

for frac in MALICIOUS_FRACTIONS:
    n_mal = int(NUM_CLIENTS * frac)
    torch.manual_seed(42)
    np.random.seed(42)
    name = f'model_poison_{int(frac*100)}pct'
    results[name] = run_experiment(
        client_data, input_dim, NUM_ROUNDS, LOCAL_EPOCHS,
        malicious_ids=list(range(n_mal)), attack='model', exp_name=name
    )

In [ ]:
# DATA POISONING
print("\n" + "="*70)
print("DATA POISONING")
print("="*70)

for frac in MALICIOUS_FRACTIONS:
    n_mal = int(NUM_CLIENTS * frac)
    torch.manual_seed(42)
    np.random.seed(42)
    name = f'data_poison_{int(frac*100)}pct'
    results[name] = run_experiment(
        client_data, input_dim, NUM_ROUNDS, LOCAL_EPOCHS,
        malicious_ids=list(range(n_mal)), attack='data', exp_name=name
    )

## 8. Results

In [ ]:
print("\n" + "="*100)
print("RESULTS SUMMARY")
print("="*100)

print(f"\n{'Experiment':<25} {'Accuracy':>12} {'F1 Score':>12} {'Precision':>12} {'Recall':>12} {'Loss':>12}")
print("-"*100)
for n, r in results.items():
    m = r.metrics
    print(f"{n:<25} {m.accuracy:>12.4f} {m.f1_score:>12.4f} {m.precision:>12.4f} {m.recall:>12.4f} {m.loss:>12.4f}")

base = results['baseline'].metrics
print("\n" + "-"*100)
print("DROPS FROM BASELINE")
print("-"*100)
print(f"\n{'Experiment':<25} {'Acc Drop':>12} {'F1 Drop':>12}")
print("-"*100)
for n, r in results.items():
    if n == 'baseline': continue
    m = r.metrics
    print(f"{n:<25} {base.accuracy-m.accuracy:>+12.4f} {base.f1_score-m.f1_score:>+12.4f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
names = list(results.keys())
x = np.arange(len(names))
colors = ['green' if 'baseline' in n else 'orange' if 'model' in n else 'red' for n in names]

for ax, (metric, title) in zip(axes.flat, [
    ('accuracy', 'Accuracy'), ('f1_score', 'F1 Score'),
    ('precision', 'Precision'), ('recall', 'Recall')
]):
    vals = [getattr(results[n].metrics, metric) for n in names]
    bars = ax.bar(x, vals, color=colors)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels([n.replace('_', '\n') for n in names], rotation=45, ha='right', fontsize=8)
    ax.axhline(y=getattr(base, metric), color='green', linestyle='--', alpha=0.7)
    ax.set_ylim(0, 1)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{v:.2f}', ha='center', fontsize=8)

plt.suptitle('Flower FL: Poisoning Attack Impact', fontsize=14)
plt.tight_layout()
plt.savefig('flower_poisoning_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for name, r in results.items():
    rounds = [h['round'] for h in r.history]
    accs = [h['accuracy'] for h in r.history]
    f1s = [h['f1_score'] for h in r.history]
    ls = '-' if 'baseline' in name else '--' if 'model' in name else ':'
    lw = 2.5 if 'baseline' in name else 1.5
    axes[0].plot(rounds, accs, label=name, linestyle=ls, linewidth=lw)
    axes[1].plot(rounds, f1s, label=name, linestyle=ls, linewidth=lw)

axes[0].set_xlabel('Round'); axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy'); axes[0].legend(fontsize=8); axes[0].grid(alpha=0.3)
axes[1].set_xlabel('Round'); axes[1].set_ylabel('F1')
axes[1].set_title('F1 Score'); axes[1].legend(fontsize=8); axes[1].grid(alpha=0.3)
plt.tight_layout()
plt.savefig('flower_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Export

In [ ]:
export = {
    'framework': 'Flower',
    'config': {
        'num_clients': NUM_CLIENTS,
        'num_rounds': NUM_ROUNDS,
        'local_epochs': LOCAL_EPOCHS,
        'malicious_fractions': MALICIOUS_FRACTIONS
    },
    'experiments': {n: r.to_dict() for n, r in results.items()},
    'baseline_metrics': results['baseline'].metrics.to_dict(),
    'communication': {
        'total_bytes': results['baseline'].comm_bytes,
        'per_round_bytes': results['baseline'].comm_bytes / NUM_ROUNDS
    },
    'poisoning_analysis': {'model_poisoning': {}, 'data_poisoning': {}}
}

for name, r in results.items():
    if 'model_poison' in name:
        pct = name.split('_')[-1]
        export['poisoning_analysis']['model_poisoning'][pct] = {
            'metrics_after': r.metrics.to_dict(),
            'accuracy_drop': base.accuracy - r.metrics.accuracy,
            'f1_drop': base.f1_score - r.metrics.f1_score
        }
    elif 'data_poison' in name:
        pct = name.split('_')[-1]
        export['poisoning_analysis']['data_poisoning'][pct] = {
            'metrics_after': r.metrics.to_dict(),
            'accuracy_drop': base.accuracy - r.metrics.accuracy,
            'f1_drop': base.f1_score - r.metrics.f1_score
        }

with open('flower_results.json', 'w') as f:
    json.dump(export, f, indent=2)
print("Saved flower_results.json")

In [ ]:
from google.colab import files
files.download('flower_results.json')
files.download('flower_poisoning_results.png')
files.download('flower_training_curves.png')
print("Done!")